In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [ ]:
# Load Train, Validation, Test Dataset
train_dataset = pd.read_csv('/kaggle/input/clinical-datasets/Google_Translated/Google_Translated/MIMIC_III/LOS_WEEKS_adm_train.csv')
val_dataset = pd.read_csv('/kaggle/input/clinical-datasets/Google_Translated/Google_Translated/MIMIC_III/LOS_WEEKS_adm_val.csv')
test_dataset = pd.read_csv('/kaggle/input/clinical-datasets/Google_Translated/Google_Translated/MIMIC_III/LOS_WEEKS_adm_test.csv')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Function to add count labels above bars
def add_count_labels(ax):
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x() + p.get_width() / 2., height + 0.5,  # Positioning the text
                f'{int(height)}', ha='center', va='bottom', fontsize=10)

# Train Set
sns.countplot(x='los_label', data=train_dataset, ax=axes[0])
axes[0].set_title("Train Dataset Class Distribution")
add_count_labels(axes[0])

# Validation Set
sns.countplot(x='los_label', data=val_dataset, ax=axes[1])
axes[1].set_title("Validation Dataset Class Distribution")
add_count_labels(axes[1])

# Test Set
sns.countplot(x='los_label', data=test_dataset, ax=axes[2])
axes[2].set_title("Test Dataset Class Distribution")
add_count_labels(axes[2])

# Display the plots
plt.tight_layout()
plt.show()

In [ ]:
from torch.utils.data import DataLoader
from torch import nn

class EnsembleModel(nn.Module):
    def __init__(self, model1):
        super(EnsembleModel, self).__init__()
        self.model1 = model1

    def forward(self, input_ids, attention_mask):
        output1 = self.model1(input_ids, attention_mask=attention_mask)[0]
        avg_output = output1
        return avg_output

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoConfig

# Specify the dropout rate in the configuration
config = AutoConfig.from_pretrained('csebuetnlp/banglabert', 
                                    num_labels=4, 
                                    hidden_dropout_prob=0.2, 
                                    attention_probs_dropout_prob=0.2)

# Load the pre-trained model with the specified configuration
core_model = AutoModelForSequenceClassification.from_pretrained('csebuetnlp/banglabert', config=config)

In [ ]:
from transformers import AutoTokenizer

# Choose a tokenizer
tokenizer = AutoTokenizer.from_pretrained('csebuetnlp/banglabert')

In [ ]:
# Apply the tokenizer to the training, validation, and test datasets
train_encodings = tokenizer(train_dataset['bn'].tolist(), truncation=True, padding=True, max_length = 512)
val_encodings = tokenizer(val_dataset['bn'].tolist(), truncation=True, padding=True,  max_length = 512)
test_encodings = tokenizer(test_dataset['bn'].tolist(), truncation=True, padding=True , max_length = 512)

In [ ]:
# Create a Dataset for PyTorch
class LosDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = LosDataset(train_encodings, train_dataset['los_label'].tolist())
val_dataset = LosDataset(val_encodings, val_dataset['los_label'].tolist())
test_dataset = LosDataset(test_encodings, test_dataset['los_label'].tolist())

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from tqdm import tqdm
from torch import nn
import numpy as np

# Create the ensemble model
ensemble_model = EnsembleModel(core_model)

In [ ]:
import os

# list all files in the current directory
files = os.listdir('.')

# filter the ones that start with 'GT_BN_BANGLA_baseline'
core_models = [f for f in files if f.startswith('GT_BN_BANGLA_baseline')]

if core_models:
    print("Found models starting with 'GT_BN_BANGLA_baseline':")
    for model in core_models:
        print(model)
        
    # get the first (and supposedly only) model
    model_path = core_models[1]

    # load the model state
    ensemble_model.load_state_dict(torch.load(model_path))
    print("Loaded Model")
else:
    print("No models found starting with 'GT_BN_BANGLA_baseline'.")

In [ ]:
# ensemble_model

In [ ]:
# Push the model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ensemble_model = ensemble_model.to(device)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
epochs = 100
best_roc_auc = 0
min_delta = 0.0001
early_stopping_count = 0
early_stopping_patience = 5
gradient_accumulation_steps = 10
best_model_path = "best_model.pth"

# Set the optimizer
optimizer = AdamW(ensemble_model.parameters(), lr=1e-5, weight_decay=0.01)

# Set the scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=50, 
    num_training_steps=len(train_loader) * epochs // gradient_accumulation_steps
)


In [ ]:
import torch
from torch.nn import functional as F
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Compute class weights
classes = np.unique(train_dataset.labels)  # Get unique class labels
class_weights = compute_class_weight('balanced', classes=classes, y=train_dataset.labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

# Print class weights with corresponding class names
print("Class Weights:")
for cls, weight in zip(classes, class_weights):
    print(f"Class {cls}: Weight {weight}")

roc_auc_values = []
# Training loop
for epoch in range(0, epochs):
    ensemble_model.train()
    train_loss = 0
    for step, batch in enumerate(tqdm(train_loader)):
        optimizer.zero_grad() if step % gradient_accumulation_steps == 0 else None
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = ensemble_model(input_ids, attention_mask)

        # Apply cost-sensitive loss
        loss_fn = nn.CrossEntropyLoss(weight=class_weights_tensor)
        loss = loss_fn(outputs, labels)

        (loss / gradient_accumulation_steps).backward()
        train_loss += loss.item()
        
        if (step + 1) % gradient_accumulation_steps == 0 or (step + 1) == len(train_loader):
            optimizer.step()
            scheduler.step()

    # Validation
    ensemble_model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = ensemble_model(input_ids, attention_mask)

            loss = loss_fn(outputs, labels)
            val_loss += loss.item()

            val_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
            val_labels.append(labels.cpu().numpy())

    val_preds = np.concatenate(val_preds)
    val_labels = np.concatenate(val_labels)
    val_loss /= len(val_loader)
    train_loss /= len(train_loader)

    print(f'Epoch: {epoch+1}/{epochs}, Training Loss: {train_loss}, Validation Loss: {val_loss}')

    # Calculate evaluation metrics
    val_preds_class = np.argmax(val_preds, axis=1)
    accuracy = accuracy_score(val_labels, val_preds_class)
    recall = recall_score(val_labels, val_preds_class, average='weighted')
    precision = precision_score(val_labels, val_preds_class, average='weighted')
    f1 = f1_score(val_labels, val_preds_class, average='weighted')
    micro_f1 = f1_score(val_labels, val_preds_class, average='micro')
    macro_roc_auc = roc_auc_score(val_labels, val_preds, multi_class='ovo', average='macro')

    print(f'Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}, Micro F1: {micro_f1}, Macro Roc Auc: {macro_roc_auc}')
    # Store the ROC-AUC value for this epoch
    roc_auc_values.append(macro_roc_auc)

    # Implement early stopping
    if epoch > 0 and macro_roc_auc - best_roc_auc < min_delta:
        early_stopping_count += 1
        print(f'EarlyStopping counter: {early_stopping_count} out of {early_stopping_patience}')
        if early_stopping_count >= early_stopping_patience:
            print('Early stopping')
            break
    else:
        best_roc_auc = macro_roc_auc
        early_stopping_count = 0
        torch.save(ensemble_model.state_dict(), f"GT_BN_BANGLA_baseline_epoch_{epoch}_roc_{best_roc_auc:.4f}.pth")

# After training, plot the ROC-AUC curve
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(roc_auc_values) + 1), roc_auc_values, marker='o', linestyle='-', color='b', label='ROC-AUC Score')

# Enhancing readability
plt.title('ROC-AUC Score Progression', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('ROC-AUC Score', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(fontsize=10, loc='lower right')
plt.ylim(0.7, 1.0)  # Adjust y-axis range if necessary

plt.show()

# Convert the list of ROC-AUC values and epochs to a DataFrame
roc_auc_df = pd.DataFrame(roc_auc_values)

# Save the DataFrame to a CSV file
roc_auc_df.to_csv('roc_auc_per_epoch.csv', index=False)

print("ROC-AUC values per epoch saved to 'roc_auc_per_epoch.csv'")

In [ ]:
import os
from torch.nn import functional as F
        
# get the first (and supposedly only) model
model_path = "/kaggle/input/bn_los_test/pytorch/default/1/GT_BN_BANGLA_baseline_epoch_5_roc_0.6882.pth"
# load the model state
ensemble_model.load_state_dict(torch.load(model_path))
print(f"Loaded Model: {model_path}")

In [ ]:
import os

# list all files in the current directory
files = os.listdir('.')

# filter the ones that start with 'GT_BN_BANGLA_baseline'
core_models = [f for f in files if f.startswith('GT_BN_BANGLA_baseline')]

if core_models:
    print("Found models starting with 'GT_BN_BANGLA_baseline':")
    for model in core_models:
        print(model)
        
    # get the first (and supposedly only) model
    model_path = core_models[1]

    # load the model state
    ensemble_model.load_state_dict(torch.load(model_path))
    print("Loaded Model")
else:
    print("No models found starting with 'GT_BN_BANGLA_baseline'.")

In [ ]:
# Put the model in evaluation mode
ensemble_model.eval()

# Initialize lists to store predictions and true labels
test_preds = []
test_labels = []

# Iterate over test data
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = ensemble_model(input_ids, attention_mask)
        test_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
        test_labels.append(labels.cpu().numpy())



In [ ]:
test_preds = np.concatenate(test_preds)
test_labels = np.concatenate(test_labels)

# Calculate metrics
test_preds_class = np.argmax(test_preds, axis=1)
accuracy = accuracy_score(test_labels, test_preds_class)
recall = recall_score(test_labels, test_preds_class, average='weighted')
precision = precision_score(test_labels, test_preds_class, average='weighted')
f1 = f1_score(test_labels, test_preds_class, average='weighted')
micro_f1 = f1_score(test_labels, test_preds_class, average='micro')
macro_roc_auc = roc_auc_score(test_labels, test_preds, multi_class='ovo', average='macro')

print(f'Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}, Micro F1: {micro_f1}, Macro Roc Auc: {macro_roc_auc}')